In [2]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

data = pd.read_csv('https://raw.githubusercontent.com/terminalai/webdev-ai/main/data/mbti.csv')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cv = CountVectorizer()

X = data["text"].values.astype(str)
y = data[["I/E", "N/S", "T/F", "P/J"]]
#print(y)
X = cv.fit_transform(X)
x = X.toarray()
x = x[:,:100]
x = torch.tensor(x)
print(x)
y = np.array(y)
y = torch.from_numpy(y)
x = x/10
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.4)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.25)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [3]:
input_size = 100
sequence_length = x_train.shape[0]
hidden_size = 256
num_layers = 2
num_classes = 4


class BDNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super(BDNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,bidirectional=True)
    self.fc = nn.Linear(hidden_size*2,num_classes)

  def forward(self,x):
    h0 = torch.zeros(self.num_layers*2,x_train.size(0),self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers*2,x_train.size(0),self.hidden_size).to(device)
    out,_ = self.lstm(x ,(h0,c0))
    out = self.fc(out[:,-1,:])
    return out

model = BDNN(input_size,hidden_size,num_layers,num_classes)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimiser = torch.optim.Adam(model.parameters(),lr=learning_rate)
iterations = 10
model.to(device)

for epoch in range(iterations+1):
  y_predicted = model(x)
  loss = criterion(y_predicted,y)
  loss.backward()
  optimiser.step()
  print('epoch: ' , epoch)
  if epoch == iterations:
    print('Training Complete')

torch.save(model,'/content/model')

RuntimeError: ignored

In [ ]:
def test(x_test,y_test):
  rand = int(input('test set no. :'))
  print('Predicted MBTI Personality Type:',model(x_test[rand]))
  print('Actual MBTI Personality Type',y_test[rand])
test(x_test, y_test)